In [13]:
import mysql.connector as mysql
import pandas as pd

In [14]:
# pip install cryptography

In [8]:
key = Fernet.generate_key()
f = Fernet(key)
token = f.encrypt(b"my deep dark secret")
token

b'gAAAAABkZOCXMtJLeOACvGP4vERsAZAXQiQDFZ3QjEE8tNNmI4s8jmbtFlius5xhTUOiumHpSEXcCbkUaeYyuqldKnrO-zJdICtspySmPiz77HVWHr_idjg='

In [33]:
import secrets

In [38]:
secrets.token_hex(16)

'd38e1f66f30d50d317a01456e290d132'

In [40]:
from cryptography.fernet import Fernet
key = b'z5fNUx1DbYoH9nVpkQvpXUsKU1nhTDyoU6n-JbAPvXo='
cipher_suite = Fernet(key)
ciphered_text = cipher_suite.encrypt(b"Bappa@1234567")   
print(ciphered_text)

b'gAAAAABkZOnncVbJbvkc1L4Hvn2mVQe3lNvPj-3T-_mEUTpd-WW8PmrF0tuqNHr6eXnovToKnr0-CkvilRQMb8DKIz0cAow5SQ=='


In [22]:
unciphered_text = (cipher_suite.decrypt(ciphered_text)).decode("utf-8")


In [12]:
import sys
import hashlib
import getpass
from passlib.hash import sha256_crypt
import MySQLdb, random, os


def SQLAddPass(username, password):
    SQL = 'insert into user values ("%s", "%s")' % (username, password)

    try:
        db = MySQLdb.connect (host='localhost', user='root', db='vedio')
        c = db.cursor()
        c.execute(SQL)
        db.commit()
        c.close()
        db.close()
        db.raw_input('Record Added - press enter to continue: ')
    except:
        print('There was a problem adding the record')
        db.raw_input ('press enter to continue')


def main(argv):


    print('\nUser & Password Storage Program v.01\n')


    username = ('Please Enter a User Name: ')
    password = sha256_crypt.encrypt(getpass.getpass('Please Enter a Password: '))

    try:
        SQLAddPass(username, password)

    except:
        sys.exit('There was a problem saving Record!')

    print('\nPassword safely stored in ' + sys.argv[1] + '\n')    

if __name__ == "__main__":
    main(sys.argv[1:])

ModuleNotFoundError: No module named 'MySQLdb'

In [24]:
db = mysql.connect(host = 'localhost', port = 3306, user = 'root', 
password= unciphered_text, 
database='Stock2023')
cursor = db.cursor()
query = "select * from mf_fund3"

cursor.execute(query)
dbs = cursor.fetchall()

In [3]:
mf_db = pd.read_sql(query, con = db)

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_33761/3875687749.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mf_db = pd.read_sql(query, con = db)


In [4]:
import mysql.connector as mysql
import pandas as pd
from datetime import datetime, timedelta,date
import warnings 
warnings.simplefilter(action='ignore')
import openpyxl

In [5]:
db = mysql.connect(host = 'localhost', port = 3306, user = 'root', password='Bappa@1234567', database='Stock2023')
cursor = db.cursor()
query = "select * from mf_fund3"

In [6]:
cursor.execute(query)
dbs = cursor.fetchall()
print(f"Data base connection successful")

Data base connection successful


In [7]:
mf_db = pd.read_sql(query, con = db)

In [8]:
mf = mf_db.loc[:,['Scheme_Code','Scheme_Name','Net_Asset_Value','Date']]
mf.dropna(axis=0 , inplace=True)

In [9]:
mf_direct= mf[mf['Scheme_Name'].str.contains('Direct')]
mf_growth = mf_direct[mf_direct['Scheme_Name'].str.contains('Growth')]
mf_growth['Net_Asset_Value'] = pd.to_numeric(mf_growth['Net_Asset_Value'], errors='coerce').fillna(0, downcast='infer')
mf_growth['Date'] = pd.to_datetime(mf_growth['Date'])

In [10]:
all_fund_ID = mf_growth.loc[(mf_growth['Date']==(pd.Timestamp(max(mf_growth['Date']))))].Scheme_Code.unique()

In [11]:
return_df = pd.DataFrame(columns=[
    'Scheme_Code', 'Scheme_Name', 'Net_Asset_Value', 'Date', '3M_OLD_Price',
    '3M_per_chng', '6M_OLD_Price', '6M_per_chng', '1Y_OLD_Price',
    '1Y_per_chng', '3Y_OLD_Price', '3Y_per_chng', '5Y_OLD_price',
    '5Y_per_chng'
])

In [13]:
count = 0

for fund in all_fund_ID:
    count = count+1
    union_liquid_mf = mf_growth[mf_growth['Scheme_Code'].str.contains(fund)]
    print(f'fatching data for {count}: {union_liquid_mf.Scheme_Name.unique()}')

    start_date = pd.Timestamp(max(union_liquid_mf['Date']))
    end_date = start_date + pd.Timedelta(days=-1825)
    # print(start_date,end_date)
    union_mf_5Y = union_liquid_mf.loc[(union_liquid_mf['Date'] <= start_date) & (union_liquid_mf['Date'] >= end_date), :]
    # print(max(union_liquid_mf['Date']),min(union_liquid_mf['Date']))

    union_mf_5Y.sort_values(['Date'],ascending=[0],inplace=True)

    union_mf_5Y['3M_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-63)
    union_mf_5Y['3M_per_chng']=((union_mf_5Y['Net_Asset_Value']/union_mf_5Y['3M_OLD_Price'])**(12/3)-1)*100
    union_mf_5Y['6M_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-126)
    union_mf_5Y['6M_per_chng']=((union_mf_5Y['Net_Asset_Value']/union_mf_5Y['6M_OLD_Price'])**(12/6)-1)*100
    union_mf_5Y['1Y_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-252)
    union_mf_5Y['1Y_per_chng']=((union_mf_5Y['Net_Asset_Value']/union_mf_5Y['1Y_OLD_Price'])**(12/12)-1)*100
    union_mf_5Y['3Y_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-756)
    union_mf_5Y['3Y_per_chng'] = ((union_mf_5Y['Net_Asset_Value']/union_mf_5Y['3Y_OLD_Price'])**(12/36)-1)*100
    union_mf_5Y['5Y_OLD_price'] = union_mf_5Y['Net_Asset_Value'].shift(-1260)
    union_mf_5Y['5Y_per_chng'] = ((union_mf_5Y['Net_Asset_Value']/union_mf_5Y['5Y_OLD_price'])**(12/60)-1)*100


fatching data for 1: ['Aditya Birla Sun Life Multi-Cap Fund-Direct Growth']
fatching data for 2: ['Aditya Birla Sun Life Frontline Equity Fund - Growth - Direct Plan']
fatching data for 3: ['Aditya Birla Sun Life Equity Advantage Fund - Growth - Direct Plan']
fatching data for 4: ['Aditya Birla Sun Life Midcap Fund - Growth - Direct Plan']
fatching data for 5: ['Aditya Birla Sun Life Small Cap Fund - Growth - Direct Plan']
fatching data for 6: ['Aditya Birla Sun Life Dividend Yield Fund - Growth - Direct Plan']
fatching data for 7: ['Aditya Birla Sun Life Pure Value Fund - Growth - Direct Plan']
fatching data for 8: ['Aditya Birla Sun Life Focused Equity Fund - Growth - Direct Plan']
fatching data for 9: ['Aditya Birla Sun Life Banking and Financial Services Fund - Direct Plan - Growth']
fatching data for 10: ['Aditya Birla Sun Life Business Cycle Fund-Direct Growth']
fatching data for 11: ['Aditya Birla Sun Life Commodity Equities Fund - Global Agri Plan - Growth - Direct Plan']
fatch

In [19]:
union_df = union_mf_5Y.iloc[:1]
    # print(union_df)
    # break
# return_df = pd.DataFrame(return_df).append(union_df,ignore_index=True)
return_df = pd.concat([return_df,union_df])

In [20]:
return_df.head()

,Scheme_Code,Scheme_Name,Net_Asset_Value,Date,3M_OLD_Price,3M_per_chng,6M_OLD_Price,6M_per_chng,1Y_OLD_Price,1Y_per_chng,3Y_OLD_Price,3Y_per_chng,5Y_OLD_price,5Y_per_chng
7210213,150346,WhiteOak Capital Flexi Cap Fund Direct Plan Gr...,10.053,2023-03-02,10.889,-27.350816,10.315,-5.015465,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
mf.dropna(axis=0 , inplace=True)

In [147]:
mf_direct= mf[mf['Scheme_Name'].str.contains('Direct')]

In [148]:
mf_growth = mf_direct[mf_direct['Scheme_Name'].str.contains('Growth')]

In [149]:
mf_growth['Scheme_Name'].value_counts()

Union Liquid Fund - Direct Plan - Growth Option                     1825
Invesco India Liquid Fund - Direct Plan - Growth                    1825
BANK OF INDIA Liquid Fund- Direct Plan- Growth                      1825
UTI-  Liquid Cash Plan - Direct Plan - Growth Option                1825
Quantum Liquid Fund - Direct Plan Growth Option                     1825
                                                                    ... 
Axis Business Cycles Fund - Direct Plan - Growth                       4
Mirae Asset Flexi Cap Fund - Direct Plan - Growth                      4
Aditya Birla Sun Life Retirement Fund-The 40s Plan-Direct Growth       1
Aditya Birla Sun Life Retirement Fund-The 50s Plan-Direct Growth       1
Aditya Birla Sun Life Retirement Fund-The 50s Plus-Direct Growth       1
Name: Scheme_Name, Length: 1297, dtype: int64

In [150]:
mf_growth['Net_Asset_Value'] = pd.to_numeric(mf_growth['Net_Asset_Value'], errors='coerce').fillna(0, downcast='infer')

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/2464115266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mf_growth['Net_Asset_Value'] = pd.to_numeric(mf_growth['Net_Asset_Value'], errors='coerce').fillna(0, downcast='infer')


In [151]:
mf_growth['Date'] = pd.to_datetime(mf_growth['Date'])

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/1839261285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mf_growth['Date'] = pd.to_datetime(mf_growth['Date'])


In [192]:
all_fund_ID = mf_growth.Scheme_Code.unique()
print(all_fund_ID)

['120704' '120706' '120572' ... '147311' '151267' '150346']


In [152]:
union_liquid_mf = mf_growth[mf_growth['Scheme_Name'].str.contains('Union Liquid Fund')]

In [153]:
union_liquid_mf.head()

,Scheme_Code,Scheme_Name,Net_Asset_Value,Date
6701254,119303,Union Liquid Fund - Direct Plan - Growth Option,1724.4805,2018-03-04
6701255,119303,Union Liquid Fund - Direct Plan - Growth Option,1724.7936,2018-03-05
6701256,119303,Union Liquid Fund - Direct Plan - Growth Option,1725.1481,2018-03-06
6701257,119303,Union Liquid Fund - Direct Plan - Growth Option,1725.5550,2018-03-07
6701258,119303,Union Liquid Fund - Direct Plan - Growth Option,1725.8655,2018-03-08


In [154]:
union_liquid_mf.shape

(1825, 4)

In [155]:
max(union_liquid_mf['Date']),min(union_liquid_mf['Date'])

(Timestamp('2023-03-02 00:00:00'), Timestamp('2018-03-04 00:00:00'))

In [156]:
from datetime import datetime, timedelta,date

In [189]:
start_date = pd.Timestamp(max(union_liquid_mf['Date']))
end_date = start_date + pd.Timedelta(days=-1825)
union_mf_5Y = union_liquid_mf.loc[(union_liquid_mf['Date'] <= start_date) & (union_liquid_mf['Date'] >= end_date), :]

In [190]:
print(start_date,  end_date)

2023-03-02 00:00:00 2018-03-03 00:00:00


In [191]:
max(union_mf_5Y['Date']),min(union_mf_5Y['Date'])

(Timestamp('2023-03-02 00:00:00'), Timestamp('2018-03-04 00:00:00'))

In [159]:
union_mf_5Y.nunique()

Scheme_Code           1
Scheme_Name           1
Net_Asset_Value    1735
Date               1735
dtype: int64

In [160]:
union_mf_5Y.sort_values(['Date'],ascending=[0],inplace=True)

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/2757731574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_mf_5Y.sort_values(['Date'],ascending=[0],inplace=True)


In [161]:
union_mf_5Y.head()

,Scheme_Code,Scheme_Name,Net_Asset_Value,Date
6703078,119303,Union Liquid Fund - Direct Plan - Growth Option,2156.1427,2023-03-02
6702987,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.4103,2022-12-01
6702986,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.0349,2022-11-30
6702985,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.6145,2022-11-29
6702984,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.1962,2022-11-28


In [162]:
union_mf_5Y['3M_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-90)

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/2320633096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_mf_5Y['3M_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-90)


In [163]:
union_mf_5Y['3M_per_chng']=(union_mf_5Y['Net_Asset_Value']/union_mf_5Y['3M_OLD_Price']-1)*100

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/2836118118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_mf_5Y['3M_per_chng']=(union_mf_5Y['Net_Asset_Value']/union_mf_5Y['3M_OLD_Price']-1)*100


In [164]:
union_mf_5Y.head()

,Scheme_Code,Scheme_Name,Net_Asset_Value,Date,3M_OLD_Price,3M_per_chng
6703078,119303,Union Liquid Fund - Direct Plan - Growth Option,2156.1427,2023-03-02,2090.7862,3.125929
6702987,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.4103,2022-12-01,2090.4754,1.479802
6702986,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.0349,2022-11-30,2090.1246,1.478874
6702985,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.6145,2022-11-29,2089.7893,1.475039
6702984,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.1962,2022-11-28,2089.4715,1.470453


In [165]:
union_mf_5Y['6M_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-120)

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/4059165019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_mf_5Y['6M_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-120)


In [166]:
union_mf_5Y['6M_per_chng']=(union_mf_5Y['Net_Asset_Value']/union_mf_5Y['6M_OLD_Price']-1)*100

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/2506056958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_mf_5Y['6M_per_chng']=(union_mf_5Y['Net_Asset_Value']/union_mf_5Y['6M_OLD_Price']-1)*100


In [167]:
union_mf_5Y.head()

,Scheme_Code,Scheme_Name,Net_Asset_Value,Date,3M_OLD_Price,3M_per_chng,6M_OLD_Price,6M_per_chng
6703078,119303,Union Liquid Fund - Direct Plan - Growth Option,2156.1427,2023-03-02,2090.7862,3.125929,2081.4058,3.590693
6702987,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.4103,2022-12-01,2090.4754,1.479802,2081.1092,1.936520
6702986,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.0349,2022-11-30,2090.1246,1.478874,2080.8100,1.933137
6702985,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.6145,2022-11-29,2089.7893,1.475039,2080.4052,1.932763
6702984,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.1962,2022-11-28,2089.4715,1.470453,2079.9503,1.934945


In [168]:
union_mf_5Y['1Y_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-365)

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/630216260.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_mf_5Y['1Y_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-365)


In [169]:
union_mf_5Y['1Y_per_chng']=(union_mf_5Y['Net_Asset_Value']/union_mf_5Y['1Y_OLD_Price']-1)*100

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/955198717.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_mf_5Y['1Y_per_chng']=(union_mf_5Y['Net_Asset_Value']/union_mf_5Y['1Y_OLD_Price']-1)*100


In [170]:
union_mf_5Y.head()

,Scheme_Code,Scheme_Name,Net_Asset_Value,Date,3M_OLD_Price,3M_per_chng,6M_OLD_Price,6M_per_chng,1Y_OLD_Price,1Y_per_chng
6703078,119303,Union Liquid Fund - Direct Plan - Growth Option,2156.1427,2023-03-02,2090.7862,3.125929,2081.4058,3.590693,2027.0197,6.370091
6702987,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.4103,2022-12-01,2090.4754,1.479802,2081.1092,1.936520,2026.8358,4.666116
6702986,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.0349,2022-11-30,2090.1246,1.478874,2080.8100,1.933137,2026.6446,4.657467
6702985,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.6145,2022-11-29,2089.7893,1.475039,2080.4052,1.932763,2026.4595,4.646281
6702984,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.1962,2022-11-28,2089.4715,1.470453,2079.9503,1.934945,2026.2769,4.635067


In [171]:
union_mf_5Y['3Y_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-1080)

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/2128541180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_mf_5Y['3Y_OLD_Price'] = union_mf_5Y['Net_Asset_Value'].shift(-1080)


In [172]:
union_mf_5Y['3Y_per_chng'] = (union_mf_5Y['Net_Asset_Value']/union_mf_5Y['3M_OLD_Price']-1)*100

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/176549941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_mf_5Y['3Y_per_chng'] = (union_mf_5Y['Net_Asset_Value']/union_mf_5Y['3M_OLD_Price']-1)*100


In [173]:
union_mf_5Y.head()

,Scheme_Code,Scheme_Name,Net_Asset_Value,Date,3M_OLD_Price,3M_per_chng,6M_OLD_Price,6M_per_chng,1Y_OLD_Price,1Y_per_chng,3Y_OLD_Price,3Y_per_chng
6703078,119303,Union Liquid Fund - Direct Plan - Growth Option,2156.1427,2023-03-02,2090.7862,3.125929,2081.4058,3.590693,2027.0197,6.370091,1879.6299,3.125929
6702987,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.4103,2022-12-01,2090.4754,1.479802,2081.1092,1.936520,2026.8358,4.666116,1879.3685,1.479802
6702986,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.0349,2022-11-30,2090.1246,1.478874,2080.8100,1.933137,2026.6446,4.657467,1879.0896,1.478874
6702985,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.6145,2022-11-29,2089.7893,1.475039,2080.4052,1.932763,2026.4595,4.646281,1878.8367,1.475039
6702984,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.1962,2022-11-28,2089.4715,1.470453,2079.9503,1.934945,2026.2769,4.635067,1878.5745,1.470453


In [174]:
union_mf_5Y['5Y_OLD_price'] = union_mf_5Y['Net_Asset_Value'].shift(-1825)

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/1961394489.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_mf_5Y['5Y_OLD_price'] = union_mf_5Y['Net_Asset_Value'].shift(-1825)


In [175]:
union_mf_5Y['%Y_per_chng'] = (union_mf_5Y['Net_Asset_Value']/union_mf_5Y['5Y_OLD_price']-1)*100

/var/folders/0y/mwh4sz4x7n98q__h0k1h0mkc0000gn/T/ipykernel_17267/1505328001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  union_mf_5Y['%Y_per_chng'] = (union_mf_5Y['Net_Asset_Value']/union_mf_5Y['5Y_OLD_price']-1)*100


In [176]:
union_mf_5Y.head()

,Scheme_Code,Scheme_Name,Net_Asset_Value,Date,3M_OLD_Price,3M_per_chng,6M_OLD_Price,6M_per_chng,1Y_OLD_Price,1Y_per_chng,3Y_OLD_Price,3Y_per_chng,5Y_OLD_price,%Y_per_chng
6703078,119303,Union Liquid Fund - Direct Plan - Growth Option,2156.1427,2023-03-02,2090.7862,3.125929,2081.4058,3.590693,2027.0197,6.370091,1879.6299,3.125929,NaN,NaN
6702987,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.4103,2022-12-01,2090.4754,1.479802,2081.1092,1.936520,2026.8358,4.666116,1879.3685,1.479802,NaN,NaN
6702986,119303,Union Liquid Fund - Direct Plan - Growth Option,2121.0349,2022-11-30,2090.1246,1.478874,2080.8100,1.933137,2026.6446,4.657467,1879.0896,1.478874,NaN,NaN
6702985,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.6145,2022-11-29,2089.7893,1.475039,2080.4052,1.932763,2026.4595,4.646281,1878.8367,1.475039,NaN,NaN
6702984,119303,Union Liquid Fund - Direct Plan - Growth Option,2120.1962,2022-11-28,2089.4715,1.470453,2079.9503,1.934945,2026.2769,4.635067,1878.5745,1.470453,NaN,NaN


In [177]:
print(union_mf_5Y.iloc[:1])

        Scheme_Code                                      Scheme_Name  \
6703078      119303  Union Liquid Fund - Direct Plan - Growth Option   

         Net_Asset_Value       Date  3M_OLD_Price  3M_per_chng  6M_OLD_Price  \
6703078        2156.1427 2023-03-02     2090.7862     3.125929     2081.4058   

         6M_per_chng  1Y_OLD_Price  1Y_per_chng  3Y_OLD_Price  3Y_per_chng  \
6703078     3.590693     2027.0197     6.370091     1879.6299     3.125929   

         5Y_OLD_price  %Y_per_chng  
6703078           NaN          NaN  


In [178]:
union_df = union_mf_5Y.iloc[:1]

In [179]:
union_df

,Scheme_Code,Scheme_Name,Net_Asset_Value,Date,3M_OLD_Price,3M_per_chng,6M_OLD_Price,6M_per_chng,1Y_OLD_Price,1Y_per_chng,3Y_OLD_Price,3Y_per_chng,5Y_OLD_price,%Y_per_chng
6703078,119303,Union Liquid Fund - Direct Plan - Growth Option,2156.1427,2023-03-02,2090.7862,3.125929,2081.4058,3.590693,2027.0197,6.370091,1879.6299,3.125929,NaN,NaN


In [193]:
union_df.columns

Index(['Scheme_Code', 'Scheme_Name', 'Net_Asset_Value', 'Date', '3M_OLD_Price',
       '3M_per_chng', '6M_OLD_Price', '6M_per_chng', '1Y_OLD_Price',
       '1Y_per_chng', '3Y_OLD_Price', '3Y_per_chng', '5Y_OLD_price',
       '%Y_per_chng'],
      dtype='object')

In [140]:
union_analysis = {}
union_analysis[union_df.Scheme_Code]={}
union_analysis[union_df.Scheme_Code]['Scheme_Name']=union_df.Scheme_Name
union_analysis[union_df.Scheme_Code]['NAV']=union_df.Net_Asset_Value
union_analysis[union_df.Scheme_Code]['3M_Ret']=union_df['3M_OLD_Price']
# union_analysis[union_df.Scheme_Code]['3M_Ret_per']=union_df['3M_per_chng']
# union_analysis[union_df.Scheme_Code]['6M_Ret']=union_df['6M_OLD_Price']
# union_analysis[union_df.Scheme_Code]['6M_Ret_per']=union_df['6M_per_chng']
# union_analysis[union_df.Scheme_Code]['1Y_Ret']=union_df['1Y_OLD_Price']
# union_analysis[union_df.Scheme_Code]['1Y_Ret_per']=union_df['1Y_per_chng']
# union_analysis[union_df.Scheme_Code]['3Y_Ret']=union_df['3Y_OLD_Price']
# union_analysis[union_df.Scheme_Code]['3Y_Ret_per']=union_df['3Y_per_chng']

TypeError: unhashable type: 'Series'